# torch.nn.Module

- 딥러닝을 구성하는 Layer의 base class이다.
- Input, Output, Forward, Backward 정의.
- 학습의 대상이 되는 parameter(tensor)정의

# nn.Parameter
- Tensor 객체의 상속 객체이다.
- nn.Moduel 내에 attribute가 될 때는 required_grad = True로 지정되어 학습 대상이 되는 Tensor이다.
- 우리가 직접 지정할 일을 거의 없다 : 대부분의 layer에는 weights 값들이 랜덤으로 지정되어 있다.

In [5]:
# Example
import torch
from torch import nn
from torch import Tensor

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        self.weights = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features))
        
    def forward(self, x : Tensor):
        return x @ self.weights + self.bias

In [6]:
x = torch.rand(5,7)
x

tensor([[0.7332, 0.7757, 0.2484, 0.3988, 0.8468, 0.2754, 0.7514],
        [0.8365, 0.3978, 0.8459, 0.7296, 0.2751, 0.7394, 0.0685],
        [0.9326, 0.5518, 0.6156, 0.4330, 0.8955, 0.1371, 0.3377],
        [0.4334, 0.5491, 0.2649, 0.6783, 0.7168, 0.8606, 0.1537],
        [0.4511, 0.9935, 0.4142, 0.4726, 0.6099, 0.4564, 0.9703]])

In [7]:
model = MyLinear(7, 12)
model(x)

tensor([[-0.1679,  1.1500, -0.2414,  2.2877,  1.3634,  1.2104, -0.5801, -1.0761,
         -0.9805, -1.7032, -0.7413, -0.2616],
        [-0.9858, -0.7790, -0.8876,  2.3111,  0.1967,  1.0963,  1.5948, -1.3491,
          1.1356,  0.8453, -1.9169,  0.0304],
        [-0.6134,  0.5531, -0.2136,  3.4132,  1.4128,  0.8275,  0.4250, -0.4719,
          0.2437, -0.3498, -1.4633, -0.6636],
        [-0.5422, -0.5667, -1.0759,  2.3869, -0.0430,  1.6814,  0.8335, -1.1540,
         -0.5942,  0.5446, -1.2121, -0.3886],
        [ 0.3170,  0.7321, -0.4113,  2.0468,  1.0104,  0.8533, -1.4176, -2.5376,
         -1.2290, -1.7428, -1.0542,  0.0621]], grad_fn=<AddBackward0>)

In [8]:
model(x).shape

torch.Size([5, 12])

In [10]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.7454,  1.1068, -0.0402, -1.0817,  1.0117,  0.2232,  1.7691,  1.5760,
           0.7920, -1.1751,  0.1174,  0.1058],
         [ 0.7390,  0.4550, -0.1149,  0.1784,  0.1279, -0.9832, -1.4799, -1.5656,
           0.2189,  0.0582, -0.2667, -0.8070],
         [ 0.3739, -1.3945, -0.2377,  2.0178,  0.3494, -1.6398,  0.1104, -1.0367,
           0.5757,  1.3210, -2.1113, -0.3058],
         [-0.2138, -0.1919,  0.0431, -0.5453, -1.2300,  1.2174,  0.2663, -1.7177,
           2.1782,  0.6055,  0.5820,  1.0517],
         [ 0.3863, -0.3415, -0.3846,  2.1309,  0.6236, -0.2094,  0.3669,  0.9587,
          -1.4542,  0.4732, -0.7349, -1.0719],
         [ 0.0999, -1.2597, -1.3965, -0.7517, -0.3921,  0.1092,  1.5462,  0.2318,
          -1.8276,  0.2938, -0.8908, -0.0254],
         [ 0.6440,  0.8575,  0.1899, -1.0899,  0.7124,  0.1493, -1.1293, -0.8793,
          -1.5656, -2.6893,  0.3315,  1.1568]], requires_grad=True),
 Parameter containing:
 tensor([-1.0409,  0.4002,  0

# Backward
- Layer에 있는 Parameter들의 미분을 수행한다.
- Forward의 결과값(model의 output => 예측치)과 실제값 간의 차이(loss)에 대해 미분을 수행한다.
- 해당 값으로 Parameter를 업데이트한다.

In [12]:
import numpy as np
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)

y_values = [2*i + 1for i in x_train]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)

In [13]:
x_train, y_train

(array([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.]], dtype=float32),
 array([[ 1.],
        [ 3.],
        [ 5.],
        [ 7.],
        [ 9.],
        [11.],
        [13.],
        [15.],
        [17.],
        [19.],
        [21.]], dtype=float32))

In [16]:
from torch.autograd import Variable

class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        
    def forward(self, x):
        output = self.linear(x)
        return output

In [41]:
inputDim = 1
outputDim = 1
learningRate = 0.01
epochs = 100

model = LinearRegression(inputDim, outputDim)

In [42]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learningRate)

In [43]:
inputs = Variable(torch.from_numpy(x_train))
labels = Variable(torch.from_numpy(y_train))
    

for epoch in range(epochs):
    
    optimizer.zero_grad()
    
    outputs = model(inputs)
    
    loss = criterion(outputs, labels)
    
    if epoch % 10 == 0:
        print(loss)
    
    loss.backward()
    
    optimizer.step()

tensor(118.5685, grad_fn=<MseLossBackward0>)
tensor(0.4350, grad_fn=<MseLossBackward0>)
tensor(0.3888, grad_fn=<MseLossBackward0>)
tensor(0.3475, grad_fn=<MseLossBackward0>)
tensor(0.3106, grad_fn=<MseLossBackward0>)
tensor(0.2776, grad_fn=<MseLossBackward0>)
tensor(0.2481, grad_fn=<MseLossBackward0>)
tensor(0.2218, grad_fn=<MseLossBackward0>)
tensor(0.1982, grad_fn=<MseLossBackward0>)
tensor(0.1771, grad_fn=<MseLossBackward0>)


In [44]:
# 실제 값과 비교해보기
y = model(Variable(torch.from_numpy(x_train)))

for i, j in zip(y, y_train):
    print(f"{i.item():.2f}, {j.item()}")

0.26, 1.0
2.36, 3.0
4.47, 5.0
6.58, 7.0
8.68, 9.0
10.79, 11.0
12.90, 13.0
15.01, 15.0
17.11, 17.0
19.22, 19.0
21.33, 21.0


# Backward from the scratch
- 실제 backward는 Module 단계에서 직접 지정가능
- Module에서 backward와 optimizer 오버라이딩
- 사용자가 직접 미분 수식을 써야하는 부담 -> 쓸 일은 없으나 순서는 이해해야한다.

# *Reference*

1. [PyTorch로 Linear Regression 하기](https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817)

2. [PyTorch로 Logistic Regression 하기](https://medium.com/dair-ai/implementing-a-logistic-regression-model-from-scratch-with-pytorch-24ea062cd856)